### `__new__()` 方法介紹

在 Python 中，`__new__()` 是一個特殊的方法，用於建立類別的實例。它在類別的實例被建立之前被呼叫，主要是用於控制物件的建立過程。這個方法的主要用途包括：

- **建立物件的實例**：`__new__()` 方法負責回傳一個新的物件實例。
- **控制物件的建立行為**：您可以在 `__new__()` 中添加邏輯來決定是否建立新物件，或是回傳已存在的物件。

#### 方法簽名

`__new__()` 方法的簽名如下：

```python
def __new__(cls, *args, **kwargs):
    # 建立一個新物件的實例
    instance = super().__new__(cls)
    return instance
```

- `cls` 參數是類別本身，與 `self` 不同，`self` 是實例本身。
- `*args` 和 `**kwargs` 使得您可以傳遞任意數量的參數。

#### 使用 `__new__()`

```python
class MyClass:
    def __new__(cls, *args, **kwargs):
        print("建立實體物件...")
        # 通過父類別的 __new__() 建立物件實例
        instance = super().__new__(cls)
        return instance

    def __init__(self, value):
        print("初始化物件...")
        self.value = value

# 建立 MyClass 的實例
obj = MyClass(10)  # 輸出：建立實體物件...
                   # 輸出：初始化物件...
```

#### `__new__()` 與 `__init__()` 的區別

- **`__new__()`**：
  - 負責建立物件的實例。
  - 回傳一個新的物件實例。
  - 當需要控制**物件建立過程**時可以改寫内容，例如**單例模式**或**不可變類型**（如元組和字串）。

- **`__init__()`**：
  - 負責初始化物件的屬性。
  - 接受已建立的物件實例( `self`參數)。
  - 在 `__new__()` 呼叫完成之後被呼叫。

#### 使用場景

1. **單例模式**：如果您希望類別只有一個實例，您可以在 `__new__()` 中檢查是否已經存在該實例，如果存在則回傳該實例。
   
2. **不可變類型**：像元組和字串這樣的不可變類型通常會重寫 `__new__()` 方法來回傳一個已經存在的實例。

### 單例模式的實作

建立一個名為 `Singleton` 的類別，確保它在整個應用程序中只有一個實例。

```python
class Singleton:
    _instance = None  # 類變數，用於儲存唯一的實例

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:  # 如果實例尚未建立
            cls._instance = super().__new__(cls)  # 建立新實例
        return cls._instance  # 回傳已存在的實例

    def __init__(self, value):
        if not hasattr(self, 'initialized'):  # 確保只初始化一次
            self.value = value
            self.initialized = True  # 設置標誌以避免再次初始化

# 使用 Singleton 類別
singleton1 = Singleton(10)
print(singleton1.value)  # 輸出：10

# 嘗試建立另一個 Singleton 實例
singleton2 = Singleton(20)
print(singleton2.value)  # 輸出：10，因為它是同一個實例

# 驗證兩個變數指向同一個實例
print(singleton1 is singleton2)  # 輸出：True
```

### 代碼解析

1. **類變數 `_instance`**：
   - `_instance` 是一個類變數，用於儲存唯一的實例。初始時設為 `None`。

2. **`__new__()` 方法**：
   - 當建立 `Singleton` 的實例時，`__new__()` 方法首先檢查 `_instance` 是否為 `None`。
   - 如果 `_instance` 為 `None`，則呼叫父類的 `__new__()` 方法來建立新實例，並將其賦值給 `_instance`。
   - 如果 `_instance` 已經存在，則直接回傳該實例。

3. **`__init__()` 方法**：
   - `__init__()` 方法在實例建立後被呼叫，用於初始化屬性。
   - 使用 `hasattr(self, 'initialized')` 來確保只初始化一次，這樣如果再次呼叫 `Singleton(20)`，不會改變已存在的實例的屬性。

### 使用範例

- 當您建立 `singleton1` 時，它初始化為 `10`。
- 當您建立 `singleton2` 並傳入 `20` 時，因為 `Singleton` 類別已經有一個實例存在，`singleton2` 實際上指向 `singleton1`，並且 `value` 仍然是 `10`。
- 最後，`singleton1 is singleton2` 回傳 `True`，確認兩個變數指向同一個實例。

### 不可變類型的實作

1. **不可變類型**：
   - 像 `int`、`str` 和 `tuple` 這些類型是不可變的，這意味著一旦它們被建立，其值就無法改變。這種不可變性是它們的一個關鍵特性，有助於在記憶體使用和性能上進行優化。

2. **`__new__` 方法**：
   - `__new__` 方法負責建立類別的實例。它在 `__init__` 之前被呼叫，通常用於那些可以以某種方式建立的實例，這些方式可能不需要每次都建立新的對象。
   - 對於不可變類型，`__new__` 可以檢查是否已經存在相同的實例。如果存在，它可以回傳現有的實例，而不是建立一個新的。
   - 通過重寫 `__new__`，像 `int`、`str` 和 `tuple` 這些類型可以實作重用現有實例的邏輯。例如，如果你建立了兩個相同的字串，Python 可以為這兩個字串回傳相同的字串對象，從而節省記憶體並提高性能。
   - 這種行為在小整數和短字串中特別明顯，Python 對它們進行緩存以優化資源使用。

為了正確實現不可變對象並確保屬性在實例建立後無法被修改，我們可以避免在 `__new__()` 方法中直接設置私有屬性。這樣會導致對 `__setattr__()` 的呼叫，進而引發 `AttributeError`。

使用字典來存儲座標並正確實現不可變性：

### 不可變類型實作範例

```python
class ImmutablePoint:
    _instances = {}  # 用於儲存已建立的實例

    def __new__(cls, x, y):
        # 使用座標作為鍵來檢查是否已經存在實例
        key = (x, y)
        if key not in cls._instances:
            # 建立新實例
            instance = super().__new__(cls)
            # 直接設置屬性
            instance.__dict__['_x'] = x
            instance.__dict__['_y'] = y
            cls._instances[key] = instance  # 儲存實例
            return instance  # 返回新實例
        return cls._instances[key]  # 返回現有的實例

    def __init__(self, x, y):
        # 初始化方法不需要做任何操作
        pass

    @property
    def x(self):
        return self._x  # 獲取私有屬性

    @property
    def y(self):
        return self._y  # 獲取私有屬性

    def __setattr__(self, name, value):
        # 禁止設置屬性，確保不可變性
        raise AttributeError(f"{name} is immutable and cannot be modified.")

# 使用 ImmutablePoint 類別
point1 = ImmutablePoint(1, 2)
print(f"Point1: ({point1.x}, {point1.y})")  # 輸出：Point1: (1, 2)

# 嘗試建立相同的 ImmutablePoint 實例
point2 = ImmutablePoint(1, 2)
print(f"Point2: ({point2.x}, {point2.y})")  # 輸出：Point2: (1, 2)

# 驗證兩個變數指向同一個實例
print(point1 is point2)  # 輸出：True，因為它們是同一個實例

# 嘗試建立不同的 ImmutablePoint 實例
point3 = ImmutablePoint(3, 4)
print(f"Point3: ({point3.x}, {point3.y})")  # 輸出：Point3: (3, 4)

# 驗證不同實例
print(point1 is point3)  # 輸出：False，因為它們是不同的實例

# 嘗試修改 x 和 y 屬性
try:
    point1.x = 5  # 嘗試修改 x
except AttributeError as e:
    print(e)  # 輸出：x is immutable and cannot be modified.

try:
    point1.y = 6  # 嘗試修改 y
except AttributeError as e:
    print(e)  # 輸出：y is immutable and cannot be modified.
```

### 代碼解析

1. **使用 `instance.__dict__`**：
   - 在 `__new__()` 方法中，我們使用 `instance.__dict__` 來設置屬性 `_x` 和 `_y`。這樣可以避免調用 `__setattr__()`，從而避免引發 `AttributeError`。

2. **私有屬性**：
   - 使用 `_x` 和 `_y` 作為屬性名稱，這樣可以避免名稱修飾的問題。

3. **不可變性**：
   - `__setattr__()` 方法禁止對任何屬性的修改，確保這些屬性不可變。

In [ ]:
class ImmutablePoint:
    _instances = {}  # 用於儲存已建立的實例

    def __new__(cls, x, y):
        # 使用座標作為鍵來檢查是否已經存在實例
        key = (x, y)
        if key not in cls._instances:
            # 建立新實例
            instance = super().__new__(cls)
            # 直接設置屬性
            instance.__dict__['_x'] = x
            instance.__dict__['_y'] = y
            cls._instances[key] = instance  # 儲存實例
            return instance  # 回傳新實例
        return cls._instances[key]  # 回傳現有的實例

    def __init__(self, x, y):
        # 初始化方法不需要做任何操作
        pass

    @property
    def x(self):
        return self._x  # 獲取私有屬性

    @property
    def y(self):
        return self._y  # 獲取私有屬性

    def __setattr__(self, name, value):
        # 禁止設置屬性，確保不可變性
        raise AttributeError(f"{name} is immutable and cannot be modified.")

# 使用 ImmutablePoint 類別
point1 = ImmutablePoint(1, 2)
print(f"Point1: ({point1.x}, {point1.y})")  # 輸出：Point1: (1, 2)

# 嘗試建立相同的 ImmutablePoint 實例
point2 = ImmutablePoint(1, 2)
print(f"Point2: ({point2.x}, {point2.y})")  # 輸出：Point2: (1, 2)

# 驗證兩個變數指向同一個實例
print(point1 is point2)  # 輸出：True，因為它們是同一個實例

# 嘗試建立不同的 ImmutablePoint 實例
point3 = ImmutablePoint(3, 4)
print(f"Point3: ({point3.x}, {point3.y})")  # 輸出：Point3: (3, 4)

# 驗證不同實例
print(point1 is point3)  # 輸出：False，因為它們是不同的實例

# 嘗試修改 x 和 y 屬性
try:
    point1.x = 5  # 嘗試修改 x
except AttributeError as e:
    print(e)  # 輸出：x is immutable and cannot be modified.

try:
    point1.y = 6  # 嘗試修改 y
except AttributeError as e:
    print(e)  # 輸出：y is immutable and cannot be modified.

### `__dict__` 屬性

在 Python 中，`__dict__` 是一個內建屬性，用於存儲類別的可寫屬性，並以字典格式表示。這個字典包含了物件的所有實例變數（屬性）及其對應的值。

1. **實例屬性**：
   - 對於類別的實例，`instance.__dict__` 會包含該實例所定義的所有屬性。例如：
     ```python
     class MyClass:
         def __init__(self, value):
             self.value = value

     obj = MyClass(10)
     print(obj.__dict__)  # 輸出: {'value': 10}
     ```

2. **類別屬性**：
   - 對於類別本身，`ClassName.__dict__` 會包含類級別的屬性和方法，包括方法定義和任何類變數：
     ```python
     class MyClass:
         class_var = 10

         def method(self):
             pass

     print(MyClass.__dict__)
     # 輸出: {'__module__': '__main__', 'class_var': 10, 'method': <function MyClass.method at ...>, ...}
     ```

3. **可寫屬性**：
   - 存儲在 `__dict__` 中的屬性可以被動態修改。您可以使用這個字典來添加、修改或刪除屬性。

4. **自定義**：
   - 您可以通過重寫 `__setattr__()` 和 `__getattr__()` 等方法來自定義屬性的訪問和修改行為。

5. **繼承**：
   - 繼承的屬性不會出現在實例的 `__dict__` 中，除非它們被重寫。Python 將會按照方法解析順序（MRO）來查找父類中的屬性。

### 範例

這是一個簡單的範例，展示 `__dict__` 的用法：

```python
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

# 創建 Person 類的實例
person = Person("Alice", 30)

# 訪問 __dict__ 屬性
print(person.__dict__)  # 輸出: {'name': 'Alice', 'age': 30}

# 透過 __dict__ 修改屬性
person.__dict__['age'] = 31
print(person.age)  # 輸出: 31
```